In [11]:
%load_ext nb_black

import os
import sys

sys.path.append(
    os.path.abspath("/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/codebase")
)
import torch.backends.cudnn as cudnn
import random
from datetime import datetime
import time
from collections import OrderedDict
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import one_hot
import utils

from Explainer.loss_F import loss_fn_kd, entropy_loss
from Explainer.models.Gated_Logic_Net import Gated_Logic_Net
from Explainer.models.explainer import Explainer
from Explainer.models.pi import Pi
from dataset.dataset_cubs import Dataset_cub_for_explainer
from dataset.utils_dataset import get_dataset_with_image_and_attributes
from Explainer.loss_F import loss_fn_kd, entropy_loss, Selective_Distillation_Loss
from Explainer.models.concepts import Conceptizator
import jupyter_notebook.stats_t as stats
import matplotlib.pyplot as plt
import warnings
import sklearn.metrics as metrics

# from ipython_utils import cal_acc, show_results
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.stats import percentileofscore
from sklearn.metrics import roc_auc_score, accuracy_score

warnings.filterwarnings("ignore")

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [12]:
disease = "edema"
path = f"/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/{disease}"
test_attr = torch.load(os.path.join(path, "dataset_g", "test_attributes.pt"))
test_proba_concept = torch.load(os.path.join(path, "dataset_g", "test_proba_concepts.pt"))
test_logits_concept = torch.load(os.path.join(path, "dataset_g", "test_logits_concepts.pt"))
test_logits_bb = torch.load(os.path.join(path, "dataset_g", "test_logits_bb.pt"))
test_class_labels = torch.load(os.path.join(path, "dataset_g", "test_class_labels.pt"))

train_attr = torch.load(os.path.join(path, "dataset_g", "train_attributes.pt"))
train_proba_concept = torch.load(os.path.join(path, "dataset_g", "train_proba_concepts.pt"))
train_logits_concept = torch.load(os.path.join(path, "dataset_g", "train_logits_concepts.pt"))
train_logits_bb = torch.load(os.path.join(path, "dataset_g", "train_logits_bb.pt"))
train_class_labels = torch.load(os.path.join(path, "dataset_g", "train_class_labels.pt"))

val_attr = torch.load(os.path.join(path, "dataset_g", "val_attributes.pt"))
val_proba_concept = torch.load(os.path.join(path, "dataset_g", "val_proba_concepts.pt"))
val_logits_concept = torch.load(os.path.join(path, "dataset_g", "val_logits_concepts.pt"))
val_logits_bb = torch.load(os.path.join(path, "dataset_g", "val_logits_bb.pt"))
val_class_labels = torch.load(os.path.join(path, "dataset_g", "val_class_labels.pt"))

print(
    test_attr.size(), test_proba_concept.size(), test_logits_concept.size(),
    test_logits_bb.size(), test_class_labels.size()
)
print()
print(
    train_attr.size(), train_proba_concept.size(), train_logits_concept.size(),
    train_logits_bb.size(), train_class_labels.size()
)
print()
print(
    val_attr.size(), val_proba_concept.size(), val_logits_concept.size(),
    val_logits_bb.size(), val_class_labels.size()
)

with open(os.path.join(path, "MIMIC_test_configs.pkl"), "rb") as input_file:
    args = pickle.load(input_file)

concepts = args.landmark_names_spec + args.abnorm_obs_concepts


torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([24072, 107]) torch.Size([48144]) torch.Size([24072, 1])

torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([187548, 107]) torch.Size([375096]) torch.Size([187548, 1])

torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([23449, 107]) torch.Size([46898]) torch.Size([23449, 1])


<IPython.core.display.Javascript object>

In [13]:
disease_index = concepts.index(disease)
concepts.index(disease), concepts[concepts.index(disease)]

(48, 'edema')

<IPython.core.display.Javascript object>

In [14]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
sub_auroc = []
sub_recall = []
all_auroc = []
cnt = 0
for i in range(len(concepts)):
    auroc_val = metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i])
    recall_val = metrics.recall_score(data_GT[:, i], y_pred[:, i])
    all_auroc.append(auroc_val)
    if auroc_val >= 0.7 and i != disease_index:
        print(f"{concepts[i]}({i}), auroc: {auroc_val}, recall: {recall_val}")
        sub_auroc.append(auroc_val)
        sub_recall.append(recall_val)
        cnt += 1

print(cnt)
np.mean(np.array(sub_auroc)), np.mean(np.array(sub_recall))

hilar_unspec(3), auroc: 0.7548264928683963, recall: 0.6856677524429967
left_pleural(4), auroc: 0.7898239824739404, recall: 0.6860189573459715
right_pleural(5), auroc: 0.8216085275479434, recall: 0.7864293659621802
pleural_unspec(6), auroc: 0.7895973042923078, recall: 0.7037773359840954
heart_size(7), auroc: 0.7343946977411204, recall: 0.6097906055461234
left_diaphragm(9), auroc: 0.7471800547075651, recall: 0.5277108433734939
right_diaphragm(10), auroc: 0.8079553421264044, recall: 0.7345309381237525
retrocardiac(12), auroc: 0.7001144370115993, recall: 0.8329604772557793
left_apical_lung(21), auroc: 0.7201363793098767, recall: 0.46774193548387094
left_lung_unspec(22), auroc: 0.7135861857839831, recall: 0.6643159379407616
right_lower_lung(23), auroc: 0.7100129396778411, recall: 0.566402378592666
right_lung_unspec(27), auroc: 0.7172979244536121, recall: 0.7053854276663146
lung_bases(29), auroc: 0.7043281098654929, recall: 0.7372368041534468
mediastinal(34), auroc: 0.7163623337591216, recal

(0.7681052731001593, 0.6786348193155134)

<IPython.core.display.Javascript object>

In [15]:
data_GT = test_attr.cpu().numpy()
data_PRED = test_proba_concept.cpu().numpy()
y_pred = np.where(data_PRED > 0.5, 1, 0)
auroc = []
recall = []
for i in range(len(concepts)):
    try:
        auroc.append(metrics.roc_auc_score(data_GT[:, i], data_PRED[:, i]))
        recall.append(metrics.recall_score(data_GT[:, i], y_pred[:, i]))
    except ValueError:
        pass

print(len(auroc))

mean_auroc = np.mean(np.array(auroc))
mean_recall = np.mean(np.array(recall))
mean_auroc, mean_recall

107


(0.7179512478289677, 0.6085155989907404)

<IPython.core.display.Javascript object>

In [16]:
mean_auroc = np.mean(np.array(auroc))
mean_recall = np.mean(np.array(recall))
mean_auroc, mean_recall

(0.7179512478289677, 0.6085155989907404)

<IPython.core.display.Javascript object>

In [17]:
mask_select = np.array(all_auroc) >= 0.7
mask_select[disease_index] = False

torch_mask = torch.from_numpy(mask_select)
torch_mask.sum(), torch_mask.size()

(tensor(62), torch.Size([107]))

<IPython.core.display.Javascript object>

In [18]:
cnt = 0
th = 0.7
concept_select = []

test_select_attr = []
test_select_proba_concept = []
test_select_logits_concept = []


val_select_attr = []
val_select_proba_concept = []
val_select_logits_concept = []


train_select_attr = []
train_select_proba_concept = []
train_select_logits_concept = []


test_select_proba_concept = []
for i in range(len(concepts)):
    if auroc[i] >= th and i != disease_index:
        test_select_attr.append(test_attr[:, i])
        test_select_proba_concept.append(test_proba_concept[:, i])
        test_select_logits_concept.append(test_logits_concept[:, i])

        val_select_attr.append(val_attr[:, i])
        val_select_proba_concept.append(val_proba_concept[:, i])
        val_select_logits_concept.append(val_logits_concept[:, i])

        train_select_attr.append(train_attr[:, i])
        train_select_proba_concept.append(train_proba_concept[:, i])
        train_select_logits_concept.append(train_logits_concept[:, i])

        concept_select.append(concepts[i])
        cnt += 1

test_select_attr = torch.stack(test_select_attr, dim=1)
test_select_proba_concept = torch.stack(test_select_proba_concept, dim=1)
test_select_logits_concept = torch.stack(test_select_logits_concept, dim=1)
val_select_attr = torch.stack(val_select_attr, dim=1)
val_select_proba_concept = torch.stack(val_select_proba_concept, dim=1)
val_select_logits_concept = torch.stack(val_select_logits_concept, dim=1)
train_select_attr = torch.stack(train_select_attr, dim=1)
train_select_proba_concept = torch.stack(train_select_proba_concept, dim=1)
train_select_logits_concept = torch.stack(train_select_logits_concept, dim=1)


print("Test sizes: ")
print(test_select_attr.size())
print(test_select_proba_concept.size())
print(test_select_logits_concept.size())
print("Val sizes: ")
print(val_select_attr.size())
print(val_select_proba_concept.size())
print(val_select_logits_concept.size())
print("Train sizes: ")
print(train_select_attr.size())
print(train_select_proba_concept.size())
print(train_select_logits_concept.size())
print(cnt)

Test sizes: 
torch.Size([24072, 62])
torch.Size([24072, 62])
torch.Size([24072, 62])
Val sizes: 
torch.Size([23449, 62])
torch.Size([23449, 62])
torch.Size([23449, 62])
Train sizes: 
torch.Size([187548, 62])
torch.Size([187548, 62])
torch.Size([187548, 62])
62


<IPython.core.display.Javascript object>

In [21]:
metric = "auroc"

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"torch_concepts_mask_{metric}.pt"),
    tensor_to_save=torch_mask,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_{metric}_attributes.pt"),
    tensor_to_save=test_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_proba_{metric}_concepts.pt"),
    tensor_to_save=test_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"test_select_logits_{metric}_concepts.pt"),
    tensor_to_save=test_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_{metric}_attributes.pt"),
    tensor_to_save=val_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_proba_{metric}_concepts.pt"),
    tensor_to_save=val_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"val_select_logits_{metric}_concepts.pt"),
    tensor_to_save=val_select_logits_concept,
)

utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_{metric}_attributes.pt"),
    tensor_to_save=train_select_attr,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_proba_{metric}_concepts.pt"),
    tensor_to_save=train_select_proba_concept,
)
utils.save_tensor(
    path=os.path.join(path, "dataset_g", f"train_select_logits_{metric}_concepts.pt"),
    tensor_to_save=train_select_logits_concept,
)

pickle.dump(
    concept_select,
    open(os.path.join(path, "dataset_g", f"selected_concepts_{metric}.pkl"), "wb"),
)
print(os.path.join(path, "dataset_g"))

/ocean/projects/asc170022p/shg121/PhD/ICLR-2022/out/mimic_cxr/t/lr_0.01_epochs_60_loss_BCE_W_flattening_type_flatten_layer_features_denseblock4/densenet121/edema/dataset_g


<IPython.core.display.Javascript object>

In [10]:
concept_select

['hilar_unspec',
 'left_pleural',
 'right_pleural',
 'pleural_unspec',
 'heart_size',
 'left_diaphragm',
 'right_diaphragm',
 'retrocardiac',
 'left_apical_lung',
 'left_lung_unspec',
 'right_lower_lung',
 'right_lung_unspec',
 'lung_bases',
 'mediastinal',
 'right_atrium',
 'right_ventricle',
 'aorta',
 'svc',
 'interstitium',
 'parenchymal',
 'stomach',
 'effusion',
 'atelectasis',
 'tube',
 'consolidation',
 'process',
 'abnormality',
 'enlarge',
 'tip',
 'low',
 'line',
 'congestion',
 'catheter',
 'cardiomegaly',
 'air',
 'tortuous',
 'lead',
 'disease',
 'calcification',
 'device',
 'engorgement',
 'elevation',
 'wire',
 'fluid',
 'degenerative',
 'pacemaker',
 'marking',
 'scar',
 'widen',
 'collapse',
 'emphysema',
 'crowd',
 'infiltrate',
 'deformity',
 'hernia',
 'drainage',
 'distention',
 'shift',
 'pressure',
 'chf',
 'redistribution',
 'aspiration']

<IPython.core.display.Javascript object>

In [18]:
y_gt_tr = torch.load(os.path.join(path, "dataset_g", f"train_class_labels.pt"))
(y_gt_tr == 1).sum(dim=0), (y_gt_tr == 0).sum(dim=0)

(tensor([21120]), tensor([166428]))

<IPython.core.display.Javascript object>

In [19]:
((y_gt_tr == 1).sum(dim=0) / y_gt_tr.size(0)).item()

0.11261117458343506

<IPython.core.display.Javascript object>